In [10]:
from numpy import mean, std
import json
from sigfig import round
hpo_scores = json.load(open('hpo_scores.json'))
for key in hpo_scores:
    avg = mean(hpo_scores[key]["error"]["measurements"])
    err = std(hpo_scores[key]["error"]["measurements"])
    hpo_scores[key]["error"]["value"], hpo_scores[key]["error"]["+/-"] = round(avg, err, cutoff=99, sep=tuple)
    hpo_scores[key]["error"]["worst_case"] = max(hpo_scores[key]["error"]["measurements"])
    hpo_scores[key]["error"]["worst_avg"] = hpo_scores[key]["error"]["value"] + hpo_scores[key]["error"]["+/-"]
json.dump(hpo_scores, open("hpo_scores_fixed.json", "w"))


In [24]:
optimal_xgb_params = {
    "n_estimators": 1000,
    "learning_rate": 0.05,
    "max_depth": 3,
    "gamma": 0,
    "colsample_bytree": 1,
    "min_child_weight": 4,
    "subsample": 1,
}

import pandas as pd
from numpy import mean, std
import json
from sigfig import round
from collections import defaultdict
scores = json.load(open('hpo_scores.json'))
set_type = lambda x: float(x) if '.' in x else int(x)
scores = {tuple(map(set_type, k[1:-1].split(", "))): v for k, v in scores.items()}

besties = set()

best_worst_error = sorted([(k, v['error']['worst_case']) for k, v in scores.items()], key=lambda x: x[1])[:3]
best_worst_error = defaultdict(lambda: "", zip([k for k, _ in best_worst_error], ("🥇", "🥈", "🥉")))
besties |= set([k for k in best_worst_error])
best_average_error = sorted([(k, v['error']['worst_avg']) for k, v in scores.items()], key=lambda x: x[1])[:3]
best_average_error = defaultdict(lambda: "", zip([k for k, _ in best_average_error], ("🥇", "🥈", "🥉")))
besties |= set([k for k in best_average_error])
min_error = min([round(mean(v['error']['measurements']), std(v['error']['measurements']), sep=tuple)[0] for v in scores.values()])
best_errors = [(k, round(mean(v['error']['measurements']), std(v['error']['measurements']), sep=tuple)) for k, v in scores.items() if round(mean(v['error']['measurements']), std(v['error']['measurements']), sep=tuple)[0] == min_error]
besties |= set([k for k, _ in best_errors])
besties = sorted([
    dict(zip(optimal_xgb_params.keys(), k)) |
    dict(zip(("error", "+/-"), round(mean(scores[k]['error']['measurements']), std(scores[k]['error']['measurements']), sep=tuple))) | {
    "worst_case": scores[k]['error']['worst_case'],
    "🥇": best_worst_error[k],
    "worst_avg": scores[k]['error']['worst_avg'],
    "🥈": best_average_error[k],
    "sorting": mean(scores[k]['error']['measurements']) + scores[k]['error']['worst_case'],
} for k in besties], key=lambda x: x['sorting'])
df = pd.DataFrame(besties).drop('sorting', axis=1)
df

,n_estimators,learning_rate,max_depth,gamma,colsample_bytree,min_child_weight,subsample,error,+/-,worst_case,🥇,worst_avg,🥈
0,1000,0.05,3,0,1,5,1,0.120,0.009,0.142044,🥇,0.1299,
1,1000,0.10,2,0,1,4,1,0.120,0.009,0.142411,🥈,0.1296,
2,1000,0.10,2,0,1,5,1,0.120,0.009,0.143100,,0.1289,🥇
3,1000,0.05,2,0,1,5,1,0.120,0.010,0.145355,,0.1291,
4,1000,0.05,3,0,1,3,1,0.120,0.010,0.143981,,0.1310,
5,1000,0.01,3,0,1,3,1,0.122,0.009,0.143046,🥉,0.1309,
6,1000,0.10,2,0,1,2,1,0.120,0.010,0.146021,,0.1310,
7,1000,0.05,2,0,1,3,1,0.120,0.009,0.147040,,0.1293,
8,1000,0.10,4,0,1,2,1,0.120,0.010,0.144445,,0.1327,
9,1000,0.05,2,0,1,4,1,0.120,0.009,0.147449,,0.1291,🥉


In [31]:
optimal_xgb_params = {k:v for k,v in df.iloc[0].to_dict().items() if k in optimal_xgb_params}
optimal_xgb_params

{'n_estimators': 1000,
 'learning_rate': 0.05,
 'max_depth': 3,
 'gamma': 0,
 'colsample_bytree': 1,
 'min_child_weight': 5,
 'subsample': 1}